#for colab

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [0]:
!pip uninstall pyglet
!pip install --user pyglet==1.3.2

Uninstalling pyglet-1.4.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/pyglet-1.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/pyglet/*
Proceed (y/n)? y
  Successfully uninstalled pyglet-1.4.1
     |████████████████████████████████| 1.0MB 6.9MB/s 


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

W0727 16:29:31.627010 140188109678464 abstractdisplay.py:151] xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
# env = wrap_env(gym.make("MsPacman-v0"))
# show_video()

#main

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pip install chainerrl

In [0]:
from __future__ import print_function
import chainer
import chainer.functions as F
import chainer.links as L
import chainerrl
import numpy as np
import gym

In [0]:
from tqdm import tqdm

In [0]:
class QFunction(chainer.Chain):
    def __init__(self):
        super(QFunction, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 16, (11, 9), 1, 0)
            self.conv2 = L.Convolution2D(16, 32, (11, 9), 1, 0)
            self.conv3 = L.Convolution2D(32, 64, (10, 9), 1, 0)
            self.l4 = L.Linear(14976, 6)
    def __call__(self, x):
        h1 = F.max_pooling_2d(F.relu(self.conv1(x)), ksize=2, stride=2)
        h2 = F.max_pooling_2d(F.relu(self.conv2(h1)), ksize=2, stride=2)
        h3 = F.max_pooling_2d(F.relu(self.conv3(h2)), ksize=2, stride=2)
        return chainerrl.action_value.DiscreteActionValue(self.l4(h3))

In [0]:
def random_action():
    return np.random.choice([0, 1, 2, 3, 4, 5])

In [0]:
gamma = 0.99
alpha = 0.5
max_number_of_steps = 20
num_episodes = 100

In [0]:
q_func = QFunction()
q_func.to_gpu(0)
optimizer = chainer.optimizers.Adam(eps=1e-3)
optimizer.setup(q_func)
explorer = chainerrl.explorers.LinearDecayEpsilonGreedy(
    start_epsilon = 1.0,
    end_epsilon = 0.1,
    decay_steps = num_episodes*100,
    random_action_func = random_action
)
replay_buffer = chainerrl.replay_buffer.ReplayBuffer(capacity=10**6)
phi = lambda x: x.astype(np.float32, copy=False)

In [0]:
agent = chainerrl.agents.DQN(
    q_func, optimizer, replay_buffer, gamma, explorer,
    replay_start_size = 5000,
    minibatch_size = 100,
    update_interval = 50,
    target_update_interval = 2000,
    phi = phi,
)

In [0]:
outdir = 'result'

In [0]:
env = wrap_env(gym.make('SpaceInvaders-v0'))
chainerrl.misc.env_modifiers.make_reward_filtered(env, lambda x: x*0.01)

In [0]:
for episode in tqdm(range(1, num_episodes+1)):
    done = False
    reward = 0
    observation = env.reset()
    observation = np.asarray(observation.transpose(2, 0, 1), dtype=np.float32) # 画像データの次元変換
    while not done:        
        action = agent.act_and_train(observation, reward)
        observation, reward, done, info = env.step(action)
        observation = np.asarray(observation.transpose(2, 0, 1), dtype=np.float32)
#         print(action, reward, done, info)
    agent.stop_episode_and_train(observation, reward, done)
    print('Episode {}: statistics: {}, epsilon {}'.format(episode, agent.get_statistics(), agent.explorer.epsilon))
    if episode % 10 == 0:
        agent.save('agent_spaceinvaders_' + str(episode))
    if episode % 10 == 0:
            env.render()
#             show_video()
#             time.sleep(20)

  1%|          | 1/100 [00:07<12:52,  7.81s/it]

Episode 1: statistics: [('average_q', 23.333140552704297), ('average_loss', 0), ('n_updates', 0)], epsilon 0.92422


  2%|▏         | 2/100 [00:10<10:24,  6.37s/it]

Episode 2: statistics: [('average_q', 30.649672593733364), ('average_loss', 0), ('n_updates', 0)], epsilon 0.87823


  3%|▎         | 3/100 [00:12<08:03,  4.99s/it]

...

Episode 35: statistics: [('average_q', 29.330712471706263), ('average_loss', 0.7207619993455492), ('n_updates', 419)], epsilon 0.1


 36%|███▌      | 36/100 [06:39<13:53, 13.02s/it]

Episode 36: statistics: [('average_q', 29.343741659045108), ('average_loss', 0.662272465932801), ('n_updates', 430)], epsilon 0.1


 37%|███▋      | 37/100 [06:53<13:58, 13.31s/it]

Episode 37: statistics: [('average_q', 29.80887066408536), ('average_loss', 0.5917796872070394), ('n_updates', 446)], epsilon 0.1
